In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd
from env import github_token, github_username

import prepare
import acquire
#import explore
#import model
from acquire import scrape_github_data
from acquire import get_github_data
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filter="ignore"

from bs4 import BeautifulSoup

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


musicbot_df = scrape_github_data()

musicbot_df

musicbot_df = pd.DataFrame(musicbot_df)
musicbot_df

In [2]:
df = get_github_data()


In [3]:
df.shape


(200, 3)

In [4]:
df.head()


,repo,language,readme_contents
0,jagrosh/MusicBot,Java,"<img align=""right"" src=""https://i.imgur.com/zr..."
1,Just-Some-Bots/MusicBot,Python,# MusicBot\n\n[![GitHub stars](https://img.shi...
2,SudhanPlayz/Discord-MusicBot,JavaScript,"<h1 align=""center""><img src=""./assets/logo.gif..."
3,IVETRI/SongPlayRoBot,Python,# Check Our New Bot Repo & Video :\n\n[Video](...
4,Splamy/TS3AudioBot,C#,# TS3AudioBot\n\nThis is a open-source TeamSpe...


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             200 non-null    object
 1   language         193 non-null    object
 2   readme_contents  186 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [6]:
df.language.value_counts()


JavaScript    73
Python        60
Java          12
TypeScript    12
Go             8
Kotlin         6
Shell          5
C#             4
Dockerfile     2
HTML           2
C++            2
Rust           2
CSS            1
PHP            1
Dart           1
Groff          1
Ruby           1
Name: language, dtype: int64

In [7]:
df = df.dropna()


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 198
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             180 non-null    object
 1   language         180 non-null    object
 2   readme_contents  180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [9]:
# prepare the data by creating clean, stemmed and lemmatized columns
df = prepare.prepare_github_data('github_df', column='readme_contents')


TypeError: string indices must be integers